<a href="https://colab.research.google.com/github/abdelrahmanAh/Sentiment-Analysis/blob/main/sentiment_Analysis_by_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import tensorflow as tf
import numpy as np


In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  12.2M      0  0:00:06  0:00:06 --:--:-- 18.2M


In [ ]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
!rm -r /content/aclImdb/train/unsup

In [ ]:
batch_size=32
rw_train_ds=tf.keras.utils.text_dataset_from_directory(
   'aclImdb/train',batch_size=batch_size,validation_split=0.2,subset='training',
    seed=1337
)
rw_val_ds=tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',batch_size=batch_size,validation_split=0.2,subset='validation',
    seed=1337
)
rw_test_ds=tf.keras.utils.text_dataset_from_directory('aclImdb/test',batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [ ]:
for text_batch,label_batch in rw_train_ds.take(1):
  for i in range(5):
    print(text_batch.numpy()[i])
    print(label_batch.numpy()[i])


b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

In [ ]:
rw_train_ds.class_names

['neg', 'pos']

In [ ]:
import string
import re

In [ ]:
def custom_standraiztion(input_data):
  lowercase=tf.strings.lower(input_data)
  stripped_html=tf.strings.regex_replace(lowercase,"<br />"," ")
  return tf.strings.regex_replace(stripped_html,f"[{re.escape(string.punctuation)}]","")

In [ ]:
max_feature=20000
embedding_dim=128
seq_len=500

In [ ]:
vectorize_layer=layers.TextVectorization(standardize=custom_standraiztion,max_tokens=max_feature,
                                         output_mode='int',output_sequence_length=seq_len)

In [ ]:
text_ds=rw_train_ds.map(lambda x,y:x)

In [ ]:
for i in text_ds.take(1):
  for n in range(2):
    print(i[n])

tf.Tensor(b'This film moved me beyond comprehension, it is and will remain my favourite film of all time, mainly because it has almost every emotion all rolled into its 157 minutes. What is the hardest part for me to take is that whenever i want to hear the amazing music and songs from the film, I have to put it into my DVD player, so I was wondering if anyone anywhere knows who sings the songs in the film and where they can be found, as I have looked everywhere I can think sporadically over the past 5 years. My favourite quote from the film is when in court the advocate says "But your own words ask for direct confrontation, isn\'t that a direct call for violence?" Biko replies "Well you and I are in confrontation now, but I see no violence!!"<br /><br />CRAIG ROBERTSON Fife, Scotland', shape=(), dtype=string)
tf.Tensor(b"Really...and incredible film that though isn't very popular...extremely touching and almost life altering...was for me at least.<br /><br />Definitely worth seeing an

In [ ]:
vectorize_layer.adapt(text_ds)

In [ ]:
def vectorize_text(text,label):
  text=tf.expand_dims(text,-1)
  return vectorize_layer(text),label

In [ ]:
train_ds=rw_train_ds.map(vectorize_text)
val_ds=rw_val_ds.map(vectorize_text)
test_ds=rw_test_ds.map(vectorize_text)

In [ ]:
train_ds=train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
test_ds=test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
val_ds=val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
inputs=layers.Input(shape=(None,),dtype='int64')
x=layers.Embedding(max_feature,embedding_dim)(inputs)
x=layers.Dropout(0.5)(x)

x=layers.Conv1D(128,7,padding='valid',activation='relu',strides=3)(x)
x=layers.Conv1D(128,7,padding='valid',activation='relu',strides=3)(x)

x=layers.GlobalMaxPooling1D()(x)

x=layers.Dense(256,activation='relu')(x)


x=layers.Dropout(0.5)(x)

prediction=layers.Dense(1,activation='sigmoid',name='predictions')(x)
model=keras.Model(inputs,prediction)
model.compile(loss=keras.losses.BinaryCrossentropy(),optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
epochs=4
model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/4
625/625 [==============================] - 114s 180ms/step - loss: 0.4862 - accuracy: 0.7225 - val_loss: 0.3506 - val_accuracy: 0.8464
Epoch 2/4
625/625 [==============================] - 113s 181ms/step - loss: 0.2172 - accuracy: 0.9135 - val_loss: 0.2966 - val_accuracy: 0.8790
Epoch 3/4
625/625 [==============================] - 118s 189ms/step - loss: 0.1138 - accuracy: 0.9597 - val_loss: 0.4509 - val_accuracy: 0.8500
Epoch 4/4
625/625 [==============================] - 137s 220ms/step - loss: 0.0636 - accuracy: 0.9773 - val_loss: 0.4482 - val_accuracy: 0.8694


In [ ]:
model.evaluate(test_ds)


782/782 [==============================] - 35s 44ms/step - loss: 0.4790 - accuracy: 0.8610


[0.47898346185684204, 0.8609600067138672]

In [ ]:
def text_analysis(text):


  vec=vectorize_layer(text)
  p=model.predict(tf.expand_dims(vec,0))
  if p>.5 :
    print(f'positive ')
  else :
    print(f'negative ')
  return p

In [ ]:
s=text_analysis('I  like the movie')
print(s)

1/1 [==============================] - 0s 24ms/step
positive 
[[0.5052487]]


In [ ]:
s=text_analysis('it is very good moviethe movie')
print(s)


1/1 [==============================] - 0s 36ms/step
positive 
[[0.97317874]]


In [ ]:
s=text_analysis('it is too much bad movie ')
print(s)


1/1 [==============================] - 0s 118ms/step
negative 
[[0.05933639]]


In [ ]:
s=text_analysis('i donot like it')
print(s)


1/1 [==============================] - 0s 28ms/step
positive 
[[0.5107164]]


In [ ]:
s=text_analysis('it is impresive ')
print(s)


1/1 [==============================] - 0s 41ms/step
positive 
[[0.50443876]]


In [ ]:
s=text_analysis('it is not bad ')
print(s)
#look because we are not use transofrmer or lstm it not recognize the context like
#not bad
#don't like
#like
#so this model is just use word classifcation and not consider the context to much
#this model it just use word like
#bad so it consider it 's negative while the context could be not bad visversa positve too

1/1 [==============================] - 0s 24ms/step
negative 
[[0.471719]]
